# Clase 3 - Disyunción
## Investigación Operativa 2C 2025 - Departamento de Matemática - FCEN-UBA

### Inicialización

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://github.com/conda-forge/miniforge/releases/download/25.3.1-0/Miniforge3-Linux-x86_64.sh")

In [ ]:
!conda install -q pyscipopt

In [1]:
# Importamos la clase Model de pyscipopt
from pyscipopt import Model
# Importamos numpy
import numpy as np

### Asignación de operarios

1) Resolver el modelo para encontrar una asignación de trabajadores a tareas de la planta de producción de MicroApple, eligiendo un valor apropiado para $M$:

$$
\begin{array}{lrlr}
\max & 0 &  & \\
\text{s.a.:} & \displaystyle\sum_{i=1}^{|T|} x_t =& N \\
  & x_t \leq& u_t & t\in T\\
  & x_t \geq & \ell_t & t\in T\\
  & x_t - x_{t+1} \leq & m_t & t\in\Gamma \\
  & x_{t+1} - x_t \leq & m_t & t\in\Gamma \\
  & x_1 \geq & 0.25x_2 - M\theta \\
  & x_3+x_4+x_5 \leq & E + M(1-\theta)\\
  & x_t\in &\mathbb{Z_{\geq 0}} & t\in T \\
  & \theta\in&\{0,1\} \\
\end{array}
$$

2) Una vez obtenida una solución, ¿cómo podrías usar el modelo para hallar una asignación distinta?  
**Sugerencia:** sea $s_t$ el valor de $x_t$ en la solución del punto 1), necesitamos que para <u>al menos</u> un $t$, valga que $x_t\neq s_t$.
Como $x_t,s_t\in\mathbb{Z}$, vale:
$$x_t\neq s_t\iff |x_t-s_t|\geq 1$$

In [2]:
# Cantidad de operarios
N = 60

# Cotas inferiores y superiores de operarios para cada trabajo
l = np.array([2, 5, 2, 4, 1, 4, 5, 1, 4, 4, 1, 3, 5, 3, 3])
u = np.array([8, 9, 8, 10, 9, 9, 8, 10, 8, 8, 7, 9, 10, 8, 8])

# Conjunto Gamma
Gamma = {3, 5, 7, 11}

# Valores de m
m = {3: 4, 5: 3, 7: 4, 11: 2}

# Valor de E
E = 8

# Valor de M
# x_2 <= 60 por ende x_2/4 <= 15, como necesito x_2/4 - M <= 0 resulta M >= 15
# x-3 + x_4 + x_5 <= 60 por ende con 60 <= E + M me alcanza, como E=8 entonces 52 <= M me sirve
# asi tomo M = MAX(15, 52) = 52
M = 52

In [3]:
model = Model('operarios')

################################################
## Definir las variables y escribir el modelo ##
################################################

# Inicializamos un vector vacío para las variables y lo llenamos con las variables
x = np.empty(len(l), dtype='object')
for i in range(len(l)):
  x[i] = model.addVar(f'x_{i}', vtype='I')

o = model.addVar('disjunction_var', vtype='B')

for i in range(len(l)):
    model.addCons(x[i] <= u[i])
    model.addCons(x[i] >= l[i])

for i in Gamma:
   model.addCons(x[i] - x[i+1] <= m[i])
   model.addCons(x[i+1] - x[i] <= m[i])

model.addCons(x[0] >= 1/4 * x[1] - M*o)
model.addCons(x[2] + x[3] + x[4] <= E + M*(1-o))

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/clasesPracticas/clase3-my.sol')

presolving:
(round 1, fast)       1 del vars, 31 del conss, 0 add conss, 30 chg bounds, 2 chg sides, 3 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       8 del vars, 33 del conss, 0 add conss, 30 chg bounds, 3 chg sides, 4 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       10 del vars, 34 del conss, 0 add conss, 30 chg bounds, 3 chg sides, 4 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 4, fast)       10 del vars, 34 del conss, 0 add conss, 42 chg bounds, 3 chg sides, 4 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 5, fast)       16 del vars, 40 del conss, 0 add conss, 42 chg bounds, 3 chg sides, 4 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (6 rounds: 6 fast, 1 medium, 1 exhaustive):
 17 deleted vars, 40 deleted constraints, 0 added constraints, 42 tightened bounds, 0 added holes, 3 changed sides, 4 changed coefficients
 0 implications, 0 cliques
transformed 1/1 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Sta

2. Para hallar una solución distinta a la que ya obtuvimos, podemos guardar la solución obtenida en un diccionario `s`, tal que `s[t]` tenga el valor de $x_t$ en la solucion obtenida con el punto 1:

In [20]:
s = [sol[x[t]] for t in range(len(l))]

In [31]:
model2 = Model('operarios_2')

#######################################################
## Volvemos a escribir el modelo, agregando las      ##
## variables y restricciones necesarias para obtener ##
## una asignacion distinta a s                       ##
#######################################################

x = np.empty(len(l), dtype='object')
for i in range(len(l)):
  x[i] = model2.addVar(f'x_{i}', vtype='I')

o = model2.addVar('disjunction_var', vtype='B')

for i in range(len(l)):
    model2.addCons(x[i] <= u[i])
    model2.addCons(x[i] >= l[i])

for i in Gamma:
   model2.addCons(x[i] - x[i+1] <= m[i])
   model2.addCons(x[i+1] - x[i] <= m[i])

model2.addCons(x[0] >= 1/4 * x[1] - M*o)
model2.addCons(x[2] + x[3] + x[4] <= E + M*(1-o))

#######################################################
## Volvemos a escribir el modelo, agregando las      ##
## variables y restricciones necesarias para obtener ##
## una asignacion distinta a s                       ##
#######################################################

# constante de disyuncion entre diferencias de la solucion nueva y la anterior
# el abs(x,s) no puede ser mas grande que 60 entonces x - s <= -1 + K resulta 60 <= -1 +  K resulta 61 <= K
# por el mismo motivo -60 >= 1 - K resulta K >= 61
K = 90

d = np.empty(2*len(l), dtype='object')
for i in range(len(l)):
   d[2*i] = model2.addVar(f'o_{2*i}', vtype='B')
   model2.addCons(x[i] - s[i] <= -1 + K*(1-d[2*i]))
   
   d[2*i + 1] = model2.addVar(f'o_{2*i + 1}', vtype='B')
   model2.addCons(x[i] - s[i] >= 1 - K*(1-d[2*i + 1]))
model2.addCons(sum(d[i] for i in range(len(d))) >= 1)

model2.redirectOutput()
model2.optimize()
sol = model2.getBestSol()
model2.writeSol(sol, './InvestigacionOperativa/clasesPracticas/clase3-my2.sol')

presolving:
(round 1, fast)       16 del vars, 46 del conss, 0 add conss, 39 chg bounds, 23 chg sides, 24 chg coeffs, 0 upgd conss, 4 impls, 0 clqs
(round 2, fast)       20 del vars, 48 del conss, 0 add conss, 39 chg bounds, 24 chg sides, 25 chg coeffs, 0 upgd conss, 4 impls, 0 clqs
(round 3, fast)       38 del vars, 63 del conss, 0 add conss, 39 chg bounds, 24 chg sides, 25 chg coeffs, 0 upgd conss, 4 impls, 0 clqs
(round 4, fast)       39 del vars, 65 del conss, 0 add conss, 39 chg bounds, 24 chg sides, 25 chg coeffs, 0 upgd conss, 4 impls, 0 clqs
(round 5, fast)       40 del vars, 65 del conss, 0 add conss, 51 chg bounds, 24 chg sides, 25 chg coeffs, 0 upgd conss, 4 impls, 0 clqs
(round 6, fast)       46 del vars, 71 del conss, 0 add conss, 51 chg bounds, 24 chg sides, 25 chg coeffs, 0 upgd conss, 4 impls, 0 clqs
presolving (7 rounds: 7 fast, 1 medium, 1 exhaustive):
 47 deleted vars, 71 deleted constraints, 0 added constraints, 51 tightened bounds, 0 added holes, 24 changed sides, 